# Dark net (YOLO_v3)

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
yolo = cv.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights") ### Download the weigts and config files from https://pjreddie.com/darknet/yolo/  and [ut them in the same directory]

In [3]:
classes = []

with open("coco.names", "r") as file:
    classes = file.read().splitlines()

In [13]:
video = cv.VideoCapture(0);

while True:
    check, frame = video.read()
    
    blob = cv.dnn.blobFromImage(frame, 1 / 255, (320, 320), (0,0,0), swapRB=True, crop=False)
    yolo.setInput(blob)
    output_layer_names = yolo.getUnconnectedOutLayersNames()
    layer_output = yolo.forward(output_layer_names)
    
    width = frame.shape[0]
    height = frame.shape[1]
    boxes = []
    confidences = []
    class_ids = []

    for output in layer_output:
        for detection in output:
            score = detection[5:]
            class_id = np.argmax(score)
            confidence = score[class_id]
            if confidence > 0.7:
                center_x = int(detection[0] * width)
                center_y = int(detection[0] * height)
                
                w = int(detection[0] * width)
                h = int(detection[0] * height)
                
                x = int(center_x-w/2)
                y = int(center_y-h/2)
                
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0,255,size=(len(boxes),3))
    if indexes.size > 0:
        for i in indexes.flatten():
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            conf = str(round(confidences[i],2))
            color = colors[i]
    
    cv.rectangle(frame, (x,y), (x+w,y+h), color, 3)
    cv.putText(frame, label + " : " + conf, (x,y), font, 1, color, 3)
    cv.imshow("Live Object Detection", frame)
    

    key = cv.waitKey(1)

    if key == ord('q'):
        break;

video.release()
cv.destroyAllWindows()